In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!rm -rf /content/dataset
!cp -r /content/drive/MyDrive/dataset /content/dataset

In [21]:
import os
DATASET_PATH = "/content/drive/MyDrive/dataset"

print("Fake images:", len(os.listdir(DATASET_PATH + "/fake")))
print("Genuine images:", len(os.listdir(DATASET_PATH + "/genuine")))

Fake images: 4312
Genuine images: 1201


In [22]:
!pip install albumentations --quiet


In [23]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split

In [24]:
IMG_SIZE = 224

def load_images(folder, label):
    images, labels = [], []
    for file in os.listdir(folder):
        img_path = os.path.join(folder, file)
        img = cv2.imread(img_path)
        if img is None:
            continue
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        img = preprocess_input(img)
        images.append(img)
        labels.append(label)
    return np.array(images), np.array(labels)

fake_imgs, fake_lbls = load_images(DATASET_PATH + "/fake", 1)
genuine_imgs, genuine_lbls = load_images(DATASET_PATH + "/genuine", 0)

X = np.concatenate([fake_imgs, genuine_imgs])
y = np.concatenate([fake_lbls, genuine_lbls])

print("Total images:", X.shape[0])

Total images: 5513


In [25]:
X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

In [26]:
base_model = EfficientNetB0(
    weights="imagenet",
    include_top=False,
    input_shape=(224,224,3)
)

base_model.trainable = False

x = GlobalAveragePooling2D()(base_model.output)
x = Dense(128, activation="relu")(x)
output = Dense(1, activation="sigmoid")(x)

model = Model(base_model.input, output)

In [27]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss="binary_crossentropy",
    metrics=[tf.keras.metrics.Precision(name="precision")]
)

In [28]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=5,
    batch_size=16,
    verbose=1
)

Epoch 1/5
276/276 ━━━━━━━━━━━━━━━━━━━━ 70s 162ms/step - loss: 0.4206 - precision: 0.8228 - val_loss: 0.2507 - val_precision: 0.9456
Epoch 2/5
276/276 ━━━━━━━━━━━━━━━━━━━━ 7s 24ms/step - loss: 0.2409 - precision: 0.9201 - val_loss: 0.1984 - val_precision: 0.9552
Epoch 3/5
276/276 ━━━━━━━━━━━━━━━━━━━━ 7s 27ms/step - loss: 0.1976 - precision: 0.9350 - val_loss: 0.1790 - val_precision: 0.9620
Epoch 4/5
276/276 ━━━━━━━━━━━━━━━━━━━━ 7s 24ms/step - loss: 0.1725 - precision: 0.9469 - val_loss: 0.1602 - val_precision: 0.9593
Epoch 5/5
276/276 ━━━━━━━━━━━━━━━━━━━━ 7s 26ms/step - loss: 0.1644 - precision: 0.9560 - val_loss: 0.1510 - val_precision: 0.9606


In [29]:
model.save("/content/drive/MyDrive/seed_model.h5")
print("Model saved ✅")

Model saved ✅


In [30]:
preds = model.predict(X_val[:10])
print(preds.flatten())

1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9s/step
[0.9868199  0.9950939  0.99710137 0.98692256 0.993566   0.99955326
 0.9998863  0.96867955 0.91733664 0.9961889 ]


In [35]:
!pip install -U gradio==4.44.0 --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.2/131.2 kB 9.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-adk 1.25.1 requires websockets<16.0.0,>=15.0.1, but you have websockets 12.0 which is incompatible.
google-genai 1.64.0 requires websockets<15.1.0,>=13.0.0, but you have websockets 12.0 which is incompatible.
dataproc-spark-connect 1.0.2 requires websockets>=14.0, but you have websockets 12.0 which is incompatible.
yfinance 0.2.66 requires websockets>=13.0, but you have websockets 12.0 which is incompatible.


In [37]:
from google.colab import files
files.download("/content/drive/MyDrive/seed_model.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>